# 蒙特卡洛积分与方差降低技术

## 1.用蒙特卡洛方法求积分

### 1.0基本思想
蒙特卡罗的方法求定积分的思路是，随机在[a,b]上取一个点x1，我们用f(x1)*(b-a)来估算曲线下方面积。然后进行对此随机采样，则其平均值就近似等于真实的积分面积(采样次数越多越接近)。整个过程如下图所示。

<div align="center">
	<img src="1.jpg" width="60%">
</div>

对于更一般的情况4次变成N次，乘以(b-a)可以看做除以 1 / (b-a)。 而前面讲过了随随机在曲线上采样因此随机到每个点的概率都是1 / (b-a)。 扩展到更一般的随机情况则有 1 / pdf(x)。 pdf(x)函数为x采用的概率密度函数。因此可以推导出更具有一般性的公式也就是蒙特卡洛积分法：

$$ F_n (x)= \frac{1}{n} \sum_{k=1}^{n} \frac{f(x_k)}{pdf(x_k)}$$

### 1.1简单情况：适用于积分区间[0,1] 

In [3]:
# 简单的蒙特卡洛求积分:exp(-x)  
m = 10000
x = runif(m)
theta.hat = mean(exp(-x))
print(theta.hat)
print(1-exp(-1)) # 实际

[1] 0.6296385
[1] 0.6321206


### 1.2对于一般区间[a,b]上的定积分

做线性变换：$y = (x-a)/(b-a)$，此时，$$x = (b-a)y+a, J^{'}=(b-a) \int_{0}^{1} g[(b-a)y+a] dy$$

进一步，如果在区间[a,b]上有$c \leq g(x) \leq d$，令$f(y) = \frac{1}{d-c} g(x)-c=\frac{1}{d-c}g[a+(b-a)y]-c$，则$0 \leq f(y) \leq 1$。

此时$$J^{'} = \int_{a}^{b}g(x) dx = S_0 J+c(b-a)$$ 其中，$S_0 = (b-a)(d-c) ,J = \int_{0}^{1}f(y)dy$。

下面简单介绍一个案例：

<div align="center">
	<img src="2.jpg" width="70%">
</div>

In [2]:
a = 2
b = 5
n = 10^4
g = function(x){exp(-x^2/2)/sqrt(2*pi)}
f = function(y){(g(a+(b-a)*y)-c)/(d-c)}
c = g(5)
d = g(2)
s_0 = (b-a)*(d-c)
x = runif(n)
y = runif(n)
J = sum(y<= f(x))/n
J_0 = s_0*J+c*(b-a)
print(J_0) # 蒙特卡洛得到的积分
integrate(g,2,5) # 精确值

[1] 0.02303637


0.02274985 with absolute error < 2.5e-16

### 1.3对于无界积分的求解

Stack上搜到一个比较满意的解释：使用一种变换将无界变为[0,1]有界：

<div align="center">
	<img src="3.jpg" width="50%">
</div>

下面是一个列题：求解标准正态分布的定积分

$$\phi = \int_{-\infty}^{x} \frac{1}{\sqrt{2 \pi} }e^{-t^2/2}dt$$

这个题我们首先将积分区域拆解:[0,x]和[0,$-\infty$].后者计算得到1/2，前者可以表示为$\frac{\theta}{\sqrt{2 \pi}}$ .对于$\theta$，我们设定变换：$y = t/x$，得到下式：

$$\theta = \int_{0}^{1} x e^{-(xy)^{2}/2} dy$$

这个积分相对来说好求的，计算完以后再代回：$\frac{1}{2}+\frac{\theta}{\sqrt{2 \pi}}$

In [1]:
x = seq(.1,2.5,length = 10)
m = 10000
u = runif(m) # 积分变量的范围在0-1
cdf = numeric(length(x))
for (i in 1:length(x)){
    g = x[i]*exp(-(u*x[i])^2/2)
    cdf[i] = mean(g)/sqrt(2*pi) + 0.5
}
Phi = pnorm(x)
print(round(rbind(x,cdf,Phi),3))

    [,1]  [,2]  [,3]  [,4]  [,5]  [,6]  [,7]  [,8]  [,9] [,10]
x   0.10 0.367 0.633 0.900 1.167 1.433 1.700 1.967 2.233 2.500
cdf 0.54 0.643 0.737 0.816 0.878 0.923 0.954 0.974 0.985 0.991
Phi 0.54 0.643 0.737 0.816 0.878 0.924 0.955 0.975 0.987 0.994


### 1.4对于来自已知分布的积分进行求解

接上一个例题，我们求来自正态分布的积分。基本思想与经验函数的定义类似，我们首先生成该已知分布的一系列随机数Z，根据$E[I(Z \leq x)]\underline{{=}}\ P(Z\leq x)=\phi (x)$公式，使频率逼近概率。

In [4]:
x = seq(.1,2.5,length = 10)
m = 10000
z = rnorm(m)
dim(x) = length(x)
p = apply(x,MARGIN = 1,FUN = function(x,z){mean(z<x)},z = z)
Phi = pnorm(x)
print(round(rbind(x,p,Phi),3)) # x是x的数值，p是蒙特卡洛积分得到的，Phi是实际的分位数

     [,1]  [,2]  [,3]  [,4]  [,5]  [,6]  [,7]  [,8]  [,9] [,10]
x   0.100 0.367 0.633 0.900 1.167 1.433 1.700 1.967 2.233 2.500
p   0.541 0.639 0.729 0.811 0.875 0.924 0.953 0.974 0.986 0.994
Phi 0.540 0.643 0.737 0.816 0.878 0.924 0.955 0.975 0.987 0.994


### 1.5计算多重积分

这个很简单，依然是使用平均值法。我们只用看一个例题：

$$\theta=\int_{0}^{1}\!\int_{0}^{1}\!e^{(x+y)^{2}}\mathrm{d}x\mathrm{d}y$$

当然，如果计算的积分区间不是[0,1]，我们使用上述提到的方法进行转换。

In [7]:
g = function(x,y) exp(x+y)^2
n = 1000000
x = runif(n)
y = runif(n)
z = g(x,y)
mean(z)

[1] 10.21224

### 1.6使用R自带的包进行检验核算

In [9]:
library("rmutil")
g<-function(x,y) exp(x+y)^2
int2(g,a=c(0,0),b=c(1,1)) # 结果十分相近


Attaching package: 'rmutil'


The following object is masked from 'package:stats':

    nobs


The following objects are masked from 'package:base':

    as.data.frame, units




[1] 10.20501

## 2.方差降低技术

### 2.0基本思想

为了使得蒙特卡洛方法中的变异性更少从而可以在仿真的时候使用更少的样本量，接下来介绍几种方差降低技术。需要回顾效率的定义，这在非参数统计学中讲过，不再赘述。


### 2.1对偶变量法

原理推导：
根据下式我们知道，当U1和U2存在负相关性时，方差会比两者独立时小：
$$V a r\left({\frac{U_{1}+U_{2}}{2}}\right)={\frac{1}{4}}(V a r(U_{1})+V a r(U_{2})+2C o v(U_{1},U_{2}))$$

我们还有定理（PPT有证明）：

如果Xi是独立的样本，f/g是单调增函数，有

$$E[f(X)g(X)]\ge\ E[f(X)]E[g(X)].$$ 

还有推论（PPT也有证明）：

如果g是单增函数，则
$Y=g(F_{X}^{-1}(U_{1}),\cdot\cdot\cdot\cdot\cdot F_{X}^{-1}(U_{n}))$和$Y^{\prime}= g(F_{X}^{-1}(1\ -U_{1}),\cdot\cdot\cdot\cdot\cdot{}f_{X}^{-1}(1-U_{n}))$呈负相关。

**需要保证g是单调函数。**

<div align="center">
	<img src="4.jpg" width="50%">
</div>

**一元积分**例题：
使用对偶变量法计算$\textstyle{\int_{0}^{1}\mathbf{e}^{-x}\mathrm{sin}\mathbf{x}\mathrm{d}\mathbf{x}}$，并与平均值法进行比较


In [15]:
n = 10000 # 需要n个重复
k = 1000 # 每次重复时随机的次数
set.seed(123) # 随机种子
MC.Phi  =  function(n,k,anti = TRUE) { # anti用来判断是否是对偶变量法，否则是普通的均值法
  f = function(x){exp(-x)*sin(x)} # 要求的函数
  theat_hat = numeric(length(k)) 
  for (i in 1:k) {
    u  =  runif(n/2)
    if (!anti) v  =  runif(n/2) else
      v  =  1 - u
    u  =  c(u, v)
    theat_hat[i] = mean(f(u))
  }
  theat_hat
}
MC1 = mean(MC.Phi(n,k))  #对偶变量积分
MC2 = mean(MC.Phi(n,k,anti=FALSE)) #平均值法积分
print(MC1)
print(MC2)
print((1-var(MC.Phi(n,k))/var(MC.Phi(n,k,anti=FALSE)))*100) #方差减少百分比
# 计算结果显示，使用对偶变量法计算的积分与均值法基本无差，但估计量的方差减少了61.93%

[1] 0.2458384
[1] 0.2458172
[1] 61.93089


**二元积分**例题：使用对偶变量法计算 $\iint_{\mathbf{D}}\,\mathbf{e}^{(\mathbf{x+y})^{2}}\mathrm{d}\mathbf{x}\mathrm{d}\mathbf{y}$，其中D = [ 0 , 1 ] × [ 0 , 1 ]，并与均值法进行比较

In [17]:
n = 10000
k = 1000
set.seed(123)
f = function(x,y){exp((x+y)^2)}

#均值法
simple_carlo = function(n,k){
  theat_hat = numeric()
  for(i in 1:k){  
    x = runif(n)
    y = runif(n)
    theat_hat[i] = mean(f(x,y))}
  return(theat_hat)
}
theat_simple = simple_carlo(n,k)
MC2 = mean(theat_simple)

#对偶变量法
MC.phi = function(n,k){
  theat_hat=numeric()
  for(i in 1:k){
    x = runif(n/4) # 变成n/4
    y = runif(n/4)
    T1 = f(x,y)
    T2 = f(x,1-y)
    T3 = f(1-x,y)
    T4 = f(1-x,1-y)
    theat_hat[i] = mean((T1+T2+T3+T4)/4)}
  return(theat_hat)
}
theat_MC = MC.phi(n,k)
MC1 = mean(theat_MC)

print(MC1)
print(MC2)
print((1-var(theat_MC)/var(theat_simple))*100)
# 从对二重积分的计算结果来看，对偶变量法依旧比平均值法减少方差约56.30%。

[1] 4.901333
[1] 4.897756
[1] 56.30473


### 2.2控制变量法

<div align="center">
	<img src="5.jpg" width="50%">
</div>

Y的选择一般不是固定的，通常与X有较强的相关性即可。

对偶变量法是控制变量法的特例。

两者相关性越强，方差减少得越多。方差的减少百分比为：
$$100\frac{[C o v(g(X),f(X))]^{2}}{V a r(g(X))\ V a r(f(X))}=100[C o r(g(X),f(X))]^{2}.$$

**一元积分**例题：计算积分$\int_{0}^{1}\,{\frac{\mathbf{e}^{-\mathbf{x}}}{1+\mathbf{x}^{2}}}\,\mathrm{d}\mathbf{x}$并与均值法比较


In [2]:
n = 10000
k = 1000
set.seed(123)
g = function(x){exp(-x)/(1+x^2)}

#均值法
simple_carlo = function(n,k){
  theat_hat = numeric()
  for(i in 1:k){  
    x = runif(n)
    theat_hat[i] = mean(g(x))}
  return(theat_hat)
}
theat_simple = simple_carlo(n,k)
MC2 = mean(theat_simple)

#控制变量法
control_carlo = function(n,k){
  f  =  function(x){exp(-.5)/(1+x^2)} # 选择的控制变量
  theat_hat = numeric()
  for(i in 1:k){
    x = runif(n)
    B  =  f(x)
    A  =  g(x)
    a  =  -cov(A,B) / var(B) # c的最小值
    theat_hat[i]  =  mean(g(x) + a * (f(x) - exp(-.5)*pi/4)) # 计算控制变量的均值 :它好像算错了，这个控制变量的期望也不好算啊
  }
  theat_hat
}
theat_control = control_carlo(n,k)
MC1 = mean(theat_control)

print(MC1)
print(MC2)
print((1-var(theat_control)/var(theat_simple))*100)
# 控制变量法的减少方差十分明显，比均值法减少了近95%的方差。

[1] 0.5247772
[1] 0.5248371
[1] 94.95615


**二元积分**例题：使用控制变量法计算 $\iint_{\mathbf{D}}\,\mathbf{e}^{(\mathbf{x+y})^{2}}\mathrm{d}\mathbf{x}\mathrm{d}\mathbf{y}$，其中D = [ 0 , 1 ] × [ 0 , 1 ]，并与均值法进行比较

In [3]:
n = 10000
k = 1000
set.seed(123)
g = function(x,y){exp((x+y)^2)}

#均值法
simple_carlo = function(n,k){
  theat_hat = numeric()
  for(i in 1:k){  
    x = runif(n)
    y = runif(n)
    theat_hat[i] = mean(g(x,y))}
  return(theat_hat)
}
theat_simple = simple_carlo(n,k)
MC2 = mean(theat_simple)

# 控制变量法
control_carlo = function(n,k){
  theat_hat = numeric()
  for(i in 1:k){
    u = runif(n)
    v = runif(n)
    f = function(u,v){exp(u+v)} # 控制变量
    r = -cov(f(u,v),g(u,v))/var(f(u,v)) #估计c
    theat_hat[i] = mean(g(u,v)+r*(f(u,v)-(exp(1)-1)^2))
  }
  return(theat_hat)
}
theat_control = control_carlo(n,k)
MC1 = mean(theat_control)

print(MC1)
print(MC2)
print((1-var(theat_control)/var(theat_simple))*100)
# 积分值约为4.9，使用控制变量法比均值法的方差减少约77.86%

[1] 4.899935
[1] 4.897756
[1] 77.86118


### 2.3重要性抽样法

<div align="center">
	<img src="6.jpg" width="50%">
</div>

均值法方法不适用于无限区间分布，并且当g不是均分分布的函数时抽样效率较低，针对这个问题提出的重要抽样方法。

$g(x)/f(x)$的方差应当较小。

**一元积分**例题：使用不同的重要性函数计算$\!\int_{0}^{1}{\frac{\mathbf{e}^{-\mathbf{x}}}{1+\mathbf{x}^{2}}}\,\mathrm{d}\mathbf{x}$

考虑如下两个重要性函数：$$\mathrm{f}_{1}(\mathbf{x})=\mathrm{e}^{-1}(1-\mathrm{e}^{-1})^{-1}$$
$$\mathrm{f}_{2}(\mathbf{x})={\frac{4}{\pi(1+\mathbf{x}^{2})}}$$

以下代码中，在生成随机数x时，用到了随机数的逆变换法。

In [7]:
n = 10000
k = 1000
set.seed(123)
g = function(x){exp(-x)/(1+x^2)}

# 均值法
simple_carlo = function(n,k){
  theta_hat = numeric()
  for(i in 1:k){  
    x = runif(n)
    theta_hat[i] = mean(g(x))}
  return(theta_hat)
}
theta_simple = simple_carlo(n,k)
MC2 = mean(theta_simple)

# 重要性函数f1
f1 = function(n,k){
  theta_hat = numeric()
  for(i in 1:k){
    u = runif(n)
    x = log(1-u*(1-exp(-1)))  # 逆变换法 从f1函数中抽取随机数
    fg = g(x)/(exp(-x)/(1-exp(-1)))  # 除以重要函数
    theta_hat[i] = mean(fg)
  }
  theta_hat
}
theta_f1 = f1(n,k)
MC11 = mean(theta_f1)

# 重要性函数f2
f2 = function(n,k){
  theta_hat = numeric()
  for(i in 1:k){
    u = runif(n)
    x = atan(pi*u/4)  # 逆变换法:不知道换的对不对，怎么感觉怪怪的
    fg = g(x)/(4/(1+x^2)*pi)
    theta_hat[i] = mean(fg)
  }
  theta_hat
}
theta_f2 = f2(n,k)
MC12 = mean(theta_f2)

print(MC11)
print(MC12)
print(MC2)
print((1-var(theta_f1)/var(theta_simple))*100)
print((1-var(theta_f2)/var(theta_simple))*100)
# 有的方法虽然方差减少的很多，但是计算结果不太一样

[1] 0.5248104
[1] 0.05659413
[1] 0.5248371
[1] 82.37536
[1] 99.78108


**二元积分**例题：使用重要抽样法计算 $\iint_{\mathbf{D}}\,\mathbf{e}^{(\mathbf{x+y})^{2}}\mathrm{d}\mathbf{x}\mathrm{d}\mathbf{y}$，其中D = [ 0 , 1 ] × [ 0 , 1 ]，并与均值法进行比较

In [8]:
#重要抽样
n = 10000
k = 1000
set.seed(123)
g = function(x,y){exp((x+y)^2)}

#均值法
simple_carlo = function(n,k){
  theat_hat = numeric()
  for(i in 1:k){  
    x = runif(n)
    y = runif(n)
    theat_hat[i] = mean(g(x,y))}
  return(theat_hat)
}
theat_simple = simple_carlo(n,k)
MC2 = mean(theat_simple)

# 重要抽样法
r_num<-function(n){ # 从函数中生成随机数
  u<-runif(n)
  x<-log((exp(1)-1)*u+1)
  return(x)
} 

importance_sample = function(n,k){
  f = function(x,y){exp(x+y)/(exp(1)-1)^2} # 选择的重要函数
  theat_hat = numeric()
  for(i in 1:k){
    x = r_num(n)
    y = r_num(n)
    fg = g(x,y)/f(x,y)
    theat_hat[i] = mean(fg)
  }
  theat_hat
}
theat_imp = importance_sample(n,k)
MC1 = mean(theat_imp)

print(MC1)
print(MC2)
print((1-var(theat_imp)/var(theat_simple))*100)
# 从结果来看，使用重要抽样比均值法的方差减少约72%

[1] 4.899765
[1] 4.897756
[1] 72.32975


### 2.4分层抽样法

<div align="center">
	<img src="7.jpg" width="60%">
</div>

过程：将区间划分为四个子区间，并使用总数的1/4计算每个子区间上积分的蒙特卡洛估计值

**一元积分**例题：$\int_{0}^{1}{\frac{e^{-x}}{1+x^{2}}}\;d x$

In [20]:
M = 10000 # 重复次数
k = 10 # 划分的区间
r = M/k # 每个区间区间重复的次数
N = 50 # 重复抽样时的次数
T2 = numeric() # 初始化
estimates = matrix(0,N,2) # 便于对比


g = function(x){
    exp(-x-log(1+x^2))*(x>0)*(x<1) # 保证不能超过[0,1]
}

for (i in 1:N){
    estimates[i,1] = mean(g(runif(M))) # 均值法
    for (j in 1:k){T2[j] = mean(g(runif(M/k,(j-1)/k,j/k)))}
    estimates[i,2] = mean(T2)
}

apply(estimates,2,mean)
apply(estimates,2,var)
print((1-apply(estimates,2,var)[2]/apply(estimates,2,var)[1])*100) # 方差降低了98%

[1] 0.5249345 0.5247806

[1] 5.138081e-06 5.146859e-08

[1] 98.99829


**二元积分**例题：试利用分层抽样法估计 $\theta = E(W_1 + W_2)^{\frac{5}{4}}$ ，其中 W1 W2 是独立同分布的weibull（威布尔）分布，其密度函数为：$f(x) = \frac{3}{2}x^{\frac{1}{2}}e^{-x^{\frac{3}{2}}}$

In [10]:
n = 10000

getU = function(){ # 实现分层抽样
  u = runif(n)
  if (n<0.5) {
    result = runif(1,min=0,max=0.5)
  } else {
    result = runif(1,min=0.5,max=1)
  }
  result
}

transF = function(u) (-log(1-u))^(2/3) # 逆变换法
 
getVal = function() {
  u = getU()
  result = floor(transF(u)) # 分段区间，向下取整
  result
}

f = function(w1,w2) (w1+w2)^(5/4) 

t10 = function(n) {
  # 初始化u1 u2
  u1 = numeric()
  u2 = numeric()
  for (i in 1:n) {
    u1[i] = getVal()
    u2[i] = getVal()
  }
  f(u1,u2)
}
 
MC1 = mean(t10(n))
print(MC1)
var(t10(n))

[1] 2.06729


[1] 1.618304

**二元积分**例题：通过分层抽样法得到 $\theta = \int^{1}_{0} \int^{1}_{0} e^{​{x+y}^2} dx dy$ 的一个模拟估计量

In [12]:
n = 10000

getVal = function() { # 分层抽样法得到随机变量
  u = runif(1)
  if (u < 0.5) {
    result = runif(1, min=0, max=0.5)
  } else {
    result = runif(1, min=0.5, max=1)
  }
  result
}
 
f = function(x,y) exp((x+y)^2)
 
t1 = function(n) {
  u1 = numeric()
  u2 = numeric()
  for (i in 1:n) {
    u1[i] = getVal()
    u2[i] = getVal()
  }
  f(u1,u2)
}
 
MC1 = mean(t1(n))
print(MC1)
var(t1(n))

[1] 4.890186


[1] 36.4484